In [1]:
import json
with open("config.json", "r") as f:
    config = json.load(f)

api_key = config["openai"]["api_key"]
model = config["openai"]["default_model"]

In [2]:
## We've got this part nailed down, I think.
from tools import tool_method, tool_manager, create_react_tool_agent

In [3]:
@tool_manager
class DictionaryToolManager:
    def __init__(self):
        self.data = {}
    
    @tool_method
    def get_value(self, key: str) -> str:
        """Get a value from the dictionary by key."""
        try:
            if key in self.data:
                value = self.data[key]
                return f"Value for key '{key}': {value}"
            else:
                return f"Key '{key}' not found in dictionary. Available keys: {list(self.data.keys())}"
        except Exception as e:
            return f"Error getting value: {str(e)}"
        
    @tool_method
    def update_value(self, key: str, value: str) -> str:
        """Update or add a key-value pair in the dictionary."""
        try:
            old_value = self.data.get(key, "not found")
            self.data[key] = value
            return f"Successfully updated key '{key}' from '{old_value}' to '{value}'"
        except Exception as e:
            return f"Error updating value: {str(e)}"
        
    @tool_method
    def list_keys(self) -> str:
        """List all keys in the dictionary."""
        try:
            keys = list(self.data.keys())
            if keys:
                return f"Dictionary keys ({len(keys)}): {keys}"
            else:
                return "Dictionary is empty (no keys found)"
        except Exception as e:
            return f"Error listing keys: {str(e)}"
        

# Example usage
my_tool_manager = DictionaryToolManager()
my_tool_manager.update_value("example_key", "example_value")

"Successfully updated key 'example_key' from 'not found' to 'example_value'"

In [ ]:
agent = create_react_tool_agent(
    model = "gpt-4.1",
    api_key = api_key,
    tools = my_tool_manager.get_tools()
)

chat_history = []
test_message = "Can you use the tools provided to get the value for 'example_key'?"
result = agent.invoke({"input": test_message, "chat_history": chat_history})
result

{'input': "Can you use the tools provided to get the value for 'example_key'?",
 'chat_history': [],
 'output': "The value for 'example_key' is: example_value.",
 'intermediate_steps': [(ToolAgentAction(tool='get_value', tool_input={'key': 'example_key'}, log="\nInvoking: `get_value` with `{'key': 'example_key'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_KyZkUg8pLnpYzXyHQVqilOv6', 'function': {'arguments': '{"key":"example_key"}', 'name': 'get_value'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-4.1-2025-04-14', 'system_fingerprint': 'fp_b3f1157249', 'service_tier': 'default'}, id='run--17d72b46-8abd-47ca-9772-f763f932e149', tool_calls=[{'name': 'get_value', 'args': {'key': 'example_key'}, 'id': 'call_KyZkUg8pLnpYzXyHQVqilOv6', 'type': 'tool_call'}], tool_call_chunks=[{'name': 'get_value', 'args': '{"key":"example_key"}', 'id': 'call_KyZkUg8pLnpYzXyHQVqilOv6', 'index': 0, 't

In [ ]:
## So what actually needs to happen here?  There exist potentially at least three different message formats:
# 1. OpenAI's message format, which is a list of dictionaries with "role" and "content" keys.
# 2. LangChain's message format, which uses classes like `HumanMessage`, `AIMessage`, and `SystemMessage`.
# 3. Chainlit's message format, which is a list of `Message` objects that can include tool calls and observations.

# What is acually required?
# - We don't actually ever need to see the OpenAI message format directly, but it does have the nice property of being primitives.
# - LangChain is what actually processes the messages.
# - ChainLit only ever sees the messages that get shown to the user, which does not include the tool calls or observations.
# - It looks like the conversions mostly just involve copying over .content
# - For LangChain, we're mostly just using HumanMessage nad AIMessage (plus the tool stuff, potentially)
# - For ChainList, we have just one message type, and we can infer the type based on whether we the app is sending or receiving it.
from langchain_core.messages.tool import ToolCall, ToolMessage
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from chainlit import Message
from typing import Union

class ChatHistoryManager:
    @staticmethod
    def chainlit_to_langchain(messages: Union[Message, list[Message]]) -> list:
        if isinstance(messages, Message):
            messages = [messages]
        # You only ever need to convert user messages
        return [HumanMessage(content=msg.content) for msg in messages if msg.type == "user"]
    
    @staticmethod
    def langchain_to_chainlit(messages: list) -> list:
        # It seems like this would mostly be about converting AssistantMessage to ChainLit Message
        # I actually don't remember; in ChainList, do we by default *see* previous user messages?
        if isinstance(messages, HumanMessage):
            messages = [messages]

    def __init__(self):
        self.history = []
    
    def add_message(self, message: str):
        self.history.append(message)

    # call this on the intermediate steps
    def parse_tool_messages(self, steps) -> list:
        # rumor has it these don't work very well
        tool_messages = []
        for step in steps:
            tool_call, tool_observation = result["intermediate_steps"][0]
            tool_call_message = ToolCall(
                name = tool_call.tool,
                args = tool_call.tool_input,
                id = tool_call.tool_call_id,
            )
            tool_message = ToolMessage(
                content = tool_observation,
                tool_call_id = tool_call.tool_call_id,
            )
            tool_messages.append(tool_call_message)
            tool_messages.append(tool_message)
        return tool_messages
    
    def get_history(self) -> list:
        return [msg for msg in self.history]

    def filter_history(self) -> list:
        # I'm not actually sure how we should do this...
        # For now, just return all messages
        return [msg for msg in self.history]


In [17]:
from langchain_core.messages.tool import ToolCall, ToolMessage
tool_call, tool_observation = result["intermediate_steps"][0]
tool_call_message = ToolCall(
    name = tool_call.tool,
    args = tool_call.tool_input,
    id = tool_call.tool_call_id,
)
tool_message = ToolMessage(
    content = tool_observation,
    tool_call_id = tool_call.tool_call_id,
)

In [19]:
help(ToolMessage)

Help on class ToolMessage in module langchain_core.messages.tool:

class ToolMessage(langchain_core.messages.base.BaseMessage, ToolOutputMixin)
 |  ToolMessage(content: Union[str, list[Union[str, dict]]], *, additional_kwargs: dict = <factory>, response_metadata: dict = <factory>, type: Literal['tool'] = 'tool', name: Optional[str] = None, id: Annotated[Optional[str], _PydanticGeneralMetadata(coerce_numbers_to_str=True)] = None, tool_call_id: str, artifact: Any = None, status: Literal['success', 'error'] = 'success', **kwargs: Any) -> None
 |  
 |  Message for passing the result of executing a tool back to a model.
 |  
 |  ToolMessages contain the result of a tool invocation. Typically, the result
 |  is encoded inside the `content` field.
 |  
 |  Example: A ToolMessage representing a result of 42 from a tool call with id
 |  
 |      .. code-block:: python
 |  
 |          from langchain_core.messages import ToolMessage
 |  
 |          ToolMessage(content='42', tool_call_id='call_J

In [18]:
tool_call_message, tool_message

({'name': 'get_value',
  'args': {'key': 'example_key'},
  'id': 'call_KyZkUg8pLnpYzXyHQVqilOv6'},
 ToolMessage(content="Value for key 'example_key': example_value", tool_call_id='call_KyZkUg8pLnpYzXyHQVqilOv6'))

In [ ]:
from langchain_core.messages.tool import ToolCall, ToolMessage

# Extract the intermediate step data
action, observation = result.intermediate_steps[0]  # First (and only) step

# Create ToolCall instance
tool_call = ToolCall(
    name=action.tool,                    # "get_value"
    args=action.tool_input,              # {"key": "example_key"}
    id=action.tool_call_id               # "call_KyZkUg8pLnpYzXyHQVqilOv6"
)

# Create ToolMessage instance
tool_message = ToolMessage(
    content=observation,                 # "Value for key 'example_key': example_value"
    tool_call_id=action.tool_call_id     # "call_KyZkUg8pLnpYzXyHQVqilOv6"
)